<a href="https://colab.research.google.com/github/nayoung718/essa_python_1/blob/main/%ED%8C%8C%EC%9D%B4%EC%8D%AC_%EB%A8%B8%EC%8B%A0%EB%9F%AC%EB%8B%9D_%EC%99%84%EB%B2%BD_%EA%B0%80%EC%9D%B4%EB%93%9C_ch8_7_pg_516_~_528.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 파이썬 머신러닝 완벽 가이드 ch8. 7 pg. 516 ~ 528

## 07. 문서 군집화 소개와 실습

- 문서 군집화 : 비슷한 텍스트 구성의 문서를 군집화

-> 텍스트 분류 기반 문서 분류는 사전에 결정 카테고리 값을 가진 학습 데이터 셋 필요, 문서 군집화는 학습 데이터 셋 필요 없는 비지도 학습 기반 동작

accuracy_garmin_nuvi_255W_gps.txt.data : garmin nuvi라는 차량용 네비게이션 모델에 대한 리뷰



In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [8]:
import pandas as pd
import glob, os

#컴퓨터에서 압축파일을 풀어놓은 디렉터리 설정
path=r'/content/drive/MyDrive/topics'
#path로 지정한 디렉터리 밑에 있는 모든 .data 파일의 파일명을 리스트로 취합
all_files=glob.glob(os.path.join(path,"*.data"))
filename_list=[]
opinion_text=[]

#개별 파일의 파일명은 filename_list로 취합,
#개별 파일의 파일 내용은 dataframe 로딩 후 다시 string으로 변환해 opinion_text list로 취합
for file_ in all_files:
  #개별 파일을 읽어서 dataframe으로 생성
  df=pd.read_table(file_, index_col=None, header=0,encoding='latin1')

  #절대 경로로 주어진 파일명 가공. 리눅스에서 수행할때는 다음 \\를 /로 변경
  #맨 마지막 .data 확장자도 제거
  filename_=file_.split('/')[-1]
  filename=filename_.split('.')[0]

  #파일명 list와 파일내용 list에 파일명과 파일 내용 추가
  filename_list.append(filename)
  opinion_text.append(df.to_string())

#파일명 list와 파일내용list 객체를 dataframe으로 생성
document_df=pd.DataFrame({'filename':filename_list, 'opinion_text':opinion_text})
document_df.head()


,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,display_garmin_nuvi_255W_gps,...
2,updates_garmin_nuvi_255W_gps,...
3,accuracy_garmin_nuvi_255W_gps,...
4,screen_ipod_nano_8gb,...


In [9]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

nltk.download('punkt')
nltk.download('wordnet')

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [10]:
# 문서 TF-IDF 형태로 피처 벡터화
# LemNormalize() 이용
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect=TfidfVectorizer(tokenizer=LemNormalize, stop_words='english',ngram_range=(1,2),min_df=0.05, max_df=0.85)


In [11]:
#opinion_text 칼럼 값으로 픽처벡터화 수행
feature_vect=tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [12]:
#kmeans수행 후 군집의 label값과 중심별로 할당된 데이터셋의 좌표 구하기
from sklearn.cluster import KMeans

#5개의 집합으록 군집화 수행
km_cluster=KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [13]:
#각 데이터별 할당된 군집의 레이블을 파일명과 파일내용 가지고 있는 df에 cluster_label 칼럼 추가해 저장
document_df['cluster_label']=cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,1
1,display_garmin_nuvi_255W_gps,...,0
2,updates_garmin_nuvi_255W_gps,...,0
3,accuracy_garmin_nuvi_255W_gps,...,0
4,screen_ipod_nano_8gb,...,1


In [14]:
#호텔 리뷰 군집화
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
3,accuracy_garmin_nuvi_255W_gps,...,0
36,buttons_amazon_kindle,...,0
10,directions_garmin_nuvi_255W_gps,...,0
1,display_garmin_nuvi_255W_gps,...,0
28,eyesight-issues_amazon_kindle,...,0
8,features_windows7,...,0
21,fonts_amazon_kindle,...,0
29,navigation_amazon_kindle,...,0
32,price_amazon_kindle,...,0
12,satellite_garmin_nuvi_255W_gps,...,0


In [15]:
#전자기기 리뷰 군집화
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
23,battery-life_amazon_kindle,...,1
0,battery-life_ipod_nano_8gb,...,1
16,battery-life_netbook_1005ha,...,1
13,keyboard_netbook_1005ha,...,1
33,performance_netbook_1005ha,...,1
4,screen_ipod_nano_8gb,...,1
11,screen_netbook_1005ha,...,1
7,size_asus_netbook_1005ha,...,1
17,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,1
5,speed_windows7,...,1


In [16]:
#차량용네비게이션 군집화
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
26,food_holiday_inn_london,...,2
35,food_swissotel_chicago,...,2
40,free_bestwestern_hotel_sfo,...,2
50,location_bestwestern_hotel_sfo,...,2
43,parking_bestwestern_hotel_sfo,...,2
37,service_bestwestern_hotel_sfo,...,2
25,service_holiday_inn_london,...,2
19,service_swissotel_hotel_chicago,...,2
39,staff_bestwestern_hotel_sfo,...,2
18,staff_swissotel_chicago,...,2


In [17]:
#호텔 리뷰 군집화
document_df[document_df['cluster_label']==3].sort_values(by='filename')

,filename,opinion_text,cluster_label
44,bathroom_bestwestern_hotel_sfo,...,3
34,location_holiday_inn_london,...,3
20,price_holiday_inn_london,...,3
31,room_holiday_inn_london,...,3
46,rooms_bestwestern_hotel_sfo,...,3
30,rooms_swissotel_chicago,...,3


In [18]:
#자동차 리뷰 군집화
document_df[document_df['cluster_label']==4].sort_values(by='filename')

,filename,opinion_text,cluster_label
48,comfort_honda_accord_2008,...,4
22,comfort_toyota_camry_2007,...,4
24,gas_mileage_toyota_camry_2007,...,4
49,interior_honda_accord_2008,...,4
27,interior_toyota_camry_2007,...,4
41,mileage_honda_accord_2008,...,4
38,performance_honda_accord_2008,...,4
47,quality_toyota_camry_2007,...,4
45,seats_honda_accord_2008,...,4
42,transmission_toyota_camry_2007,...,4


In [19]:
#군집개수 약간 많이 설정돼있어 세분화되어 군집화된 경향있음 -> 중심개수 5에서 3으로 낮춰서 3개 그룹으로 군집화한뒤 확인
from sklearn.cluster import KMeans

#3개의 집합으로 군집화
km_cluster=KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label=km_cluster.labels_
cluster_centers=km_cluster.cluster_centers_

#소속 군집을 cluster_label칼럼으로 할당하고 cluster_label 값으로 정렬
document_df['cluster_label']=cluster_label
document_df.sort_values(by='cluster_label')

/usr/local/lib/python3.10/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,0
29,navigation_amazon_kindle,...,0
28,eyesight-issues_amazon_kindle,...,0
36,buttons_amazon_kindle,...,0
23,battery-life_amazon_kindle,...,0
21,fonts_amazon_kindle,...,0
17,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,0
33,performance_netbook_1005ha,...,0
15,video_ipod_nano_8gb,...,0
14,screen_garmin_nuvi_255W_gps,...,0


In [20]:
#전자기기 리뷰 군집화
document_df[document_df['cluster_label']==0].sort_values(by='filename')

,filename,opinion_text,cluster_label
3,accuracy_garmin_nuvi_255W_gps,...,0
23,battery-life_amazon_kindle,...,0
0,battery-life_ipod_nano_8gb,...,0
16,battery-life_netbook_1005ha,...,0
36,buttons_amazon_kindle,...,0
10,directions_garmin_nuvi_255W_gps,...,0
1,display_garmin_nuvi_255W_gps,...,0
28,eyesight-issues_amazon_kindle,...,0
8,features_windows7,...,0
21,fonts_amazon_kindle,...,0


In [21]:
#호텔 리뷰 군집화
document_df[document_df['cluster_label']==1].sort_values(by='filename')

,filename,opinion_text,cluster_label
44,bathroom_bestwestern_hotel_sfo,...,1
26,food_holiday_inn_london,...,1
35,food_swissotel_chicago,...,1
40,free_bestwestern_hotel_sfo,...,1
50,location_bestwestern_hotel_sfo,...,1
34,location_holiday_inn_london,...,1
43,parking_bestwestern_hotel_sfo,...,1
20,price_holiday_inn_london,...,1
31,room_holiday_inn_london,...,1
46,rooms_bestwestern_hotel_sfo,...,1


In [22]:
#자동차 리뷰 군집화
document_df[document_df['cluster_label']==2].sort_values(by='filename')

,filename,opinion_text,cluster_label
48,comfort_honda_accord_2008,...,2
22,comfort_toyota_camry_2007,...,2
24,gas_mileage_toyota_camry_2007,...,2
49,interior_honda_accord_2008,...,2
27,interior_toyota_camry_2007,...,2
41,mileage_honda_accord_2008,...,2
38,performance_honda_accord_2008,...,2
47,quality_toyota_camry_2007,...,2
45,seats_honda_accord_2008,...,2
42,transmission_toyota_camry_2007,...,2


- 군집별 핵심 단어 추출하기

clusdters_centers_ : 각 군집을 구성하는 단어 피처가 군집의 중심을 기준으로 얼마나 가깝게 위치해있는지 제공

In [23]:
cluster_centers=km_cluster.cluster_centers_
print('cluster_centers shape: ',cluster_centers.shape)
print(cluster_centers)

cluster_centers shape:  (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]]


0 ~ 1 사이 존재, 1에 가까울수록 중심과 가까움

-> 큰 순으로 정렬된 위치 인덱스값 반환 -> 핵심 단어 피처 이름 출력

In [24]:
#군집별 top n 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명을 반환함
def get_cluster_details(cluster_model, cluster_data, cluster_nums, feature_names, top_n_features=10):

    cluster_details = {}

    #cluster_centers array의 값이 큰 순으로 정렬된 인덱스 값 반환
    #군집중심점별 할당된 word 피처들의 거리값이 큰순으로 값을 구하기 위함
    centroid_feature_ordered_ind = cluster_model.cluster_centers_ .argsort()[:, ::-1] # 행별(군집별)로 역순 정렬

    #개별 군집별로 반복하면서 핵심 단어, 그 단어의 중심 위치 상댓값, 대상 파일명 입력
    for cluster_num in range(cluster_nums):
        # 군집별 정보를 담을 데이터 초기화
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster']=cluster_num

        #cluster_centers_.argsort()[:,::-1]로 구한 인덱스를 이용해 top n 피처 단어를 구함
        top_feature_indexes=centroid_feature_ordered_ind[cluster_num, :top_n_features]
        top_features=[feature_names[ind] for ind in top_feature_indexes]

        #top_feature_indexes를 이용해 해당 피처 단어의 중심 위치 상댓값 구함
        top_feature_values=cluster_model.cluster_centers_[cluster_num, top_feature_indexes].tolist()

        #cluster_details 딕셔너리 객체에 개별 군집별 핵심 단어와 중심 위치 상댓값, 해당 파일명 입력
        cluster_details[cluster_num]['top_features']=top_features
        cluster_details[cluster_num]['top_features_value']=top_feature_values
        filenames=cluster_data[cluster_data['cluster_label']==cluster_num]['filename']
        filenames=filenames.values.tolist()

        cluster_details[cluster_num]['filenames']=filenames

    return cluster_details

In [25]:
def print_cluster_details(cluster_details):
  for cluster_num, cluster_detail in cluster_details.items():
    print('###### Cluster {0}'.format(cluster_num))
    print('Top features:', cluster_detail['top_features'])
    print('Reviews 파일명:',cluster_detail['filenames'][:7])
    print('===============================================')

In [28]:
feature_names = tfidf_vect.get_feature_names_out()
cluster_details=get_cluster_details(cluster_model=km_cluster, cluster_data=document_df, feature_names=feature_names, cluster_nums=3, top_n_features=10)
print_cluster_details(cluster_details)

###### Cluster 0
Top features: ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
Reviews 파일명: ['battery-life_ipod_nano_8gb', 'display_garmin_nuvi_255W_gps', 'updates_garmin_nuvi_255W_gps', 'accuracy_garmin_nuvi_255W_gps', 'screen_ipod_nano_8gb', 'speed_windows7', 'voice_garmin_nuvi_255W_gps']
###### Cluster 1
Top features: ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
Reviews 파일명: ['staff_swissotel_chicago', 'service_swissotel_hotel_chicago', 'price_holiday_inn_london', 'service_holiday_inn_london', 'food_holiday_inn_london', 'rooms_swissotel_chicago', 'room_holiday_inn_london']
###### Cluster 2
Top features: ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
Reviews 파일명: ['comfort_toyota_camry_2007', 'gas_mileage_toyota_camry_2007', 'interior_toyota_camry_2007', 'performance_honda_accord_2008', 'mileage_honda_a

- 포터블 전자제품 리뷰 : cluster #0 : screen, battery, life 같은 화면과 배터리 수명이 핵심 단어로 군집화됨

- 호텔 리뷰 : cluster #1 : room, hotel, service 같은 방과 서비스가 핵심 단어로 군집화됨

- 자동차 리뷰 : cluster #2 : interior, seat 같은 실내 인테리어, 좌석, 연료 효율이 핵심 단어로 군집화됨